In [ ]:
!pip install pandas

In [1]:
import datetime
from datetime import datetime as dt
import pandas as pd

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters
from google.cloud.bigtable.row_set import RowSet 

In [3]:
INSTANCE_ID = 'big-table-db2'
TABLE_GENERATION_ID = 'plant_generation'
TABLE_WEATHER_ID = 'plant_weather'

In [4]:
client = bigtable.Client.from_service_account_json('../../lofty-psyche-328203-949244c34274.json', admin = True)

In [5]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


In [6]:
print('Creating the {} table'.format(TABLE_GENERATION_ID))
table_generation = instance.table(TABLE_GENERATION_ID)

if not table_generation.exists():
    table_generation.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_GENERATION_ID))

print('Creating the {} table'.format(TABLE_WEATHER_ID))
table_weather = instance.table(TABLE_WEATHER_ID)

if not table_weather.exists():
    table_weather.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_WEATHER_ID))

Creating the plant_generation table
Creating the plant_weather table


In [7]:
print('Creating weather family columns')

plant_cfId = 'plant'
plant_cf = table_weather.column_family(plant_cfId)
plant_cf.create()

telemetry_cfId = 'telemetry'
telemetry_cf = table_weather.column_family(telemetry_cfId)
telemetry_cf.create()

Creating weather family columns


In [8]:
df = pd.read_csv('../data/Plant_2_Weather_Sensor_Data.csv')

In [9]:
print('Writing to the table')
rows = []
for line in range(len(df)):
    row_key = 'plant#{}#{}'.format(df['PLANT_ID'][line], df['DATE_TIME'][line]).encode()
    date_time = datetime.datetime.strptime(df['DATE_TIME'][line], '%Y-%m-%d %H:%M:%S')
    row = table_weather.direct_row(row_key)

    row.set_cell(plant_cfId, 'source_key'.encode(), df['SOURCE_KEY'][line], timestamp = date_time)

    row.set_cell(telemetry_cfId, 'ambient_temp'.encode(), str(df['AMBIENT_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'module_temp'.encode(), str(df['MODULE_TEMPERATURE'][line]), timestamp = date_time)
    row.set_cell(telemetry_cfId, 'irradiation'.encode(), str(df['IRRADIATION'][line]), timestamp = date_time)

    rows.append(row)

table_weather.mutate_rows(rows)

Writing to the table


[,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

In [12]:
print('Reading a range of rows')

row_set = RowSet()
row_set.add_row_range_from_keys(
    start_key = b'plant#4136001#2020-06-01 06:00:00',
    end_key = b'plant#4136001#2020-06-01 12:00:00'
)

col_filter = row_filters.ColumnQualifierRegexFilter('ambient_temp'.encode())

rows = table_weather.read_rows(row_set=row_set, filter_=col_filter)

for row in rows:
    print_row(row)

Reading a range of rows
Reading data for plant#4136001#2020-06-01 06:00:00:
Column Family telemetry
	ambient_temp: 23.470155033333327 @2020-06-01 06:00:00+00:00

Reading data for plant#4136001#2020-06-01 06:15:00:
Column Family telemetry
	ambient_temp: 23.67332510344828 @2020-06-01 06:15:00+00:00

Reading data for plant#4136001#2020-06-01 06:30:00:
Column Family telemetry
	ambient_temp: 23.979217633333327 @2020-06-01 06:30:00+00:00

Reading data for plant#4136001#2020-06-01 06:45:00:
Column Family telemetry
	ambient_temp: 24.209690400000003 @2020-06-01 06:45:00+00:00

Reading data for plant#4136001#2020-06-01 07:00:00:
Column Family telemetry
	ambient_temp: 24.567252344827587 @2020-06-01 07:00:00+00:00

Reading data for plant#4136001#2020-06-01 07:15:00:
Column Family telemetry
	ambient_temp: 24.835614033333336 @2020-06-01 07:15:00+00:00

Reading data for plant#4136001#2020-06-01 07:30:00:
Column Family telemetry
	ambient_temp: 25.36163324137931 @2020-06-01 07:30:00+00:00

Reading data

In [11]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")